# Hong Kong Weather data Preparation & Data Quality check

Here I aggregate 38 datasets coming from different observatory for daily Temperature, Humidity and Rainfall data. For each dataset, I check the number and the percentage of invalid observations (value of \*\*\*).

After aggregating, a global dataset by the information is created, so that we have **three global datasets** in the end.

In [1]:
# Necessary modules
import pandas as pd  # type: ignore
import time
import datetime
import os
import glob
import numpy as np # type: ignore

## (1) Hong Kong Maximal Temperature Dataset

In [14]:
# Public Data Downloaded from HK Data Gov website
# CSV File list in 0_Data directory:
list_files = glob.glob("../0_Data/Temperature/*.csv")

# Aggregated Data templete:
Agg_Data = pd.DataFrame()
Data_summary = pd.DataFrame()

for Data_by_dist in list_files:
    
    Max_region=pd.read_csv(Data_by_dist, 
                       sep=",", skiprows=3, header = None)

    # LMMAXT_HKP_: Full Data before any filtering
    LMMAXT_HKP_ = pd.DataFrame(Max_region)
    LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
    
    # Data Quality indicators:
    Obs_summary = pd.DataFrame({'Dist': [Data_by_dist.split("Data/Temperature/")[1].split("_")[1]],
                                'NObs_Dist': [LMMAXT_HKP_.shape[0]],
                                'NObs_Flagged': [LMMAXT_HKP_.loc[LMMAXT_HKP_['Flag']=="#"].shape[0]],
                                'NObs_Starred': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="***"].shape[0]],
                                'NObs_Str_Flagged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value']=="***")&(LMMAXT_HKP_['Flag']=="#")].shape[0]]                          
                                #'NObs_2007': LMMAXT_HKP_.loc[LMMAXT_HKP_['year']=='2007'].shape[0],
                                #'NObs_St_or_2007': LMMAXT_HKP_.loc[(LMMAXT_HKP_['year']=='2007')|(LMMAXT_HKP_['value']=="***")].shape[0]
                                })
   
    Obs_summary['Starred_Pct(%)'] = round(100*Obs_summary['NObs_Starred']/Obs_summary['NObs_Dist'],2)
    #Obs_summary['Tot_Purge_Pct(%)'] = round(100*Obs_summary['NObs_St_or_2007']/Obs_summary['NObs_Dist'],2)
        
    ## Data Filter ##
    # LMMAXT_HKP_2: Filtered Data
    # F1: Delete the last 3 lines:
    LMMAXT_HKP_2 = LMMAXT_HKP_.iloc[:-3]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'month'] = LMMAXT_HKP_2['month'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'day'] = LMMAXT_HKP_2['day'].values.astype(int)
    LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['value']!="***"]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'value'] = LMMAXT_HKP_2['value'].values.astype(float)

    # F2: Delete data of 2007 since incomplete (only from october)
    #LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['year']!='2007']
    
    # Insert the Observatory Name:
    LMMAXT_HKP_2['Dist'] = Data_by_dist.split("Data/Temperature/")[1].split("_")[1]

    # Date parsing 
    LMMAXT_HKP_date = pd.to_datetime(LMMAXT_HKP_2[['year','month', 'day']])
    #LMMAXT_HKP_date = LMMAXT_HKP_date.dt.dayofyear
    
    # Concatenate parsed date + Table 
    # LMMAXT_HKP_3: Concatenated Data
    LMMAXT_HKP_3 = pd.concat([LMMAXT_HKP_date, 
                              LMMAXT_HKP_2[['Dist','year','value', 'Flag']]],
            axis = 1)
    LMMAXT_HKP_3 = LMMAXT_HKP_3.rename(columns = {0: "Date"})
    
    
    # Information on the observed period by Observatory:
    Obs_summary['StartDate'] = LMMAXT_HKP_date.min()
    Obs_summary['EndDate'] = LMMAXT_HKP_date.max()
    
    Data_summary = pd.concat([Data_summary, Obs_summary], axis = 0)
    
    Agg_Data = pd.concat([Agg_Data, LMMAXT_HKP_3], axis = 0, ignore_index=True)

Check if all the lines are completed, computing the percentage of null values:

In [15]:
round(100*Agg_Data.loc[Agg_Data['value'].isna()!=False].shape[0]/Agg_Data.shape[0],1)

0.0

Total number of observations before filtering (From all observatories):

In [16]:
total_obs = Data_summary['NObs_Dist'].sum()
print(total_obs)

362334


Total number of observations after filtering on 2007 and the starred observations (From all observatories):

In [17]:
Agg_Data.shape[0]

358380

Checking the total number of observations, given that we applied 2 filters and 3 lines deletion containing some suppementary informations at the end of each table:

In [20]:
total_obs -Data_summary['NObs_Starred'].sum() -(3*38)

np.int64(358380)

Distribution by observatories:

In [21]:
Data_Composition = pd.DataFrame({
    'Observatory':Data_summary['Dist'].values,
    'NObs':Data_summary['NObs_Dist'].values,
    'Percentage(%)To_TotalOBS':round((Data_summary['NObs_Dist']/total_obs)*100,2),
    'ObservedPeriod(#Year)':[np.ceil(td/np.timedelta64(1, 'D')).astype(int) for td in (Data_summary['EndDate']-Data_summary['StartDate'])/(30*12)] 
})

Data_Composition.sort_values(by=['Percentage(%)To_TotalOBS','ObservedPeriod(#Year)'], 
                             ascending = False,
                             inplace = True)
Data_Composition.reset_index(inplace=True, drop = True)
print(Data_Composition.head())
print(Data_Composition.tail())

  Observatory   NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
0         HKO  49006                     13.53                    144
1         SHA  14764                      4.07                     41
2         LFS  14399                      3.97                     40
3         TKL  13425                      3.71                     38
4         HKS  12999                      3.59                     37
   Observatory  NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
33         TY1  5298                      1.46                     15
34          TW  5201                      1.44                     15
35         SE1  3730                      1.03                     11
36         YLP  3637                      1.00                     11
37         CWB  2285                      0.63                      7


In [22]:
print(round(Data_Composition['ObservedPeriod(#Year)'].mean(),1))

27.2


We see that the observatory which logged Temperature data for the longest period in Hong Kong is the one in Hong Kong Observatory in Tsim Sha Tsui area (HKO, for 144 years). It composes of about 14% of the total dataset. The shortest observated period is given by Clear Water Bay Observatory (CWB, for 7 years). The mean value of the observed period over 38 observatories is 27 years.

When we look for the percentage of the invalid lines (containing \*\*\* value), the observatory providing the highest data quality issue is Ngong Ping Observatory (5.5%). The highest invalid data ratio (5.5%) is still relatively low. It goes down to 0% (only one line deleted) when it comes to Hong Kong Observatory which is the most historical observatory. 

In [25]:
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).head())
print("...")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).tail())

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  NGP       7855           432            5.50
0  SEK      10320           439            4.25
0  WGL      12968           520            4.01
0  JKB      12147           412            3.39
0  TPO       8710           293            3.36
...
  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  TY1       5298             2            0.04
0  SSH       7520             3            0.04
0   TW       5201             2            0.04
0  WTS       5816             1            0.02
0  HKO      49006             1            0.00


Without access to the definition of these flags, I have used all numerical values including the ones flagged to '#' in Data Completeness column if it contains a normal numerical value other than '***'.

As of Temperature dataset, the percentage of observations filled with '***' takes less than 1% for 71% of 38 Hong Kong districts. 

In [26]:
round(100*Data_summary.loc[Data_summary['Starred_Pct(%)']<1].shape[0]/Data_summary.shape[0],0)

71.0

Save the aggregated Temperature data in Wrangled directory:

In [27]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
Agg_Data.to_pickle(treated_data_rep+"Temperature_AGG.pkl")

## (2) Hong Kong Mean Humidity Dataset

In [74]:
# Public Data Downloaded from HK Data Gov website
# CSV File list in 0_Data directory:
list_files = glob.glob("../0_Data/Humidity/*.csv")
    
Max_region=pd.read_csv(list_files[0], 
                    sep=",", skiprows=3, header = None)

# LMMAXT_HKP_: Full Data before any filtering
LMMAXT_HKP_ = pd.DataFrame(Max_region)
LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']

print(LMMAXT_HKP_.head())
print(LMMAXT_HKP_.tail())

   year  month  day  value Flag
0  1947    1.0  1.0   85.0    C
1  1947    1.0  2.0   86.0    C
2  1947    1.0  3.0   84.0    C
3  1947    1.0  4.0   85.0    C
4  1947    1.0  5.0   85.0    C
                          year  month   day  value Flag
28547                     2025    2.0  27.0   76.0    C
28548                     2025    2.0  28.0   77.0    C
28549     *** 沒有數據/unavailable    NaN   NaN    NaN  NaN
28550  # 數據不完整/data incomplete    NaN   NaN    NaN  NaN
28551     C 數據完整/data Complete    NaN   NaN    NaN  NaN


The structure of data seems exactly the same as Max Temperature dataset. Therefore, we proceed the wrangling with the same code. 

In [75]:

# Aggregated Data templete:
Agg_Data = pd.DataFrame()
Data_summary = pd.DataFrame()

for Data_by_dist in list_files:
    
    Max_region=pd.read_csv(Data_by_dist, 
                       sep=",", skiprows=3, header = None)

    # LMMAXT_HKP_: Full Data before any filtering
    LMMAXT_HKP_ = pd.DataFrame(Max_region)
    LMMAXT_HKP_.columns = ['year', 'month', 'day', 'value', 'Flag']
    
    # Data Quality indicators:
    Obs_summary = pd.DataFrame({'Dist': [Data_by_dist.split("Data/Humidity/")[1].split("_")[1]],
                                'NObs_Dist': [LMMAXT_HKP_.shape[0]],
                                'NObs_Flagged': [LMMAXT_HKP_.loc[LMMAXT_HKP_['Flag']=="#"].shape[0]],
                                'NObs_Starred': [LMMAXT_HKP_.loc[LMMAXT_HKP_['value']=="***"].shape[0]],
                                'NObs_Str_Flagged': [LMMAXT_HKP_.loc[(LMMAXT_HKP_['value']=="***")&(LMMAXT_HKP_['Flag']=="#")].shape[0]]                          
                                #'NObs_2007': LMMAXT_HKP_.loc[LMMAXT_HKP_['year']=='2007'].shape[0],
                                #'NObs_St_or_2007': LMMAXT_HKP_.loc[(LMMAXT_HKP_['year']=='2007')|(LMMAXT_HKP_['value']=="***")].shape[0]
                                })
   
    Obs_summary['Starred_Pct(%)'] = round(100*Obs_summary['NObs_Starred']/Obs_summary['NObs_Dist'],2)
    #Obs_summary['Tot_Purge_Pct(%)'] = round(100*Obs_summary['NObs_St_or_2007']/Obs_summary['NObs_Dist'],2)
        
    ## Data Filter ##
    # LMMAXT_HKP_2: Filtered Data
    # F1: Delete the last 3 lines:
    LMMAXT_HKP_2 = LMMAXT_HKP_.iloc[:-3]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'month'] = LMMAXT_HKP_2['month'].values.astype(int)
    LMMAXT_HKP_2.loc[:,'day'] = LMMAXT_HKP_2['day'].values.astype(int)
    LMMAXT_HKP_2 = LMMAXT_HKP_2.loc[LMMAXT_HKP_2['value']!="***"]
    LMMAXT_HKP_2.reset_index(inplace=True, drop=True) 
    LMMAXT_HKP_2.loc[:,'value'] = LMMAXT_HKP_2['value'].values.astype(float)
    
    # Insert the Observatory Name:
    LMMAXT_HKP_2['Dist'] = Data_by_dist.split("Data/Humidity/")[1].split("_")[1]

    # Date parsing 
    LMMAXT_HKP_date = pd.to_datetime(LMMAXT_HKP_2[['year','month', 'day']])
    #LMMAXT_HKP_date = LMMAXT_HKP_date.dt.dayofyear
    
    # Concatenate parsed date + Table 
    # LMMAXT_HKP_3: Concatenated Data
    LMMAXT_HKP_3 = pd.concat([LMMAXT_HKP_date, 
                              LMMAXT_HKP_2[['Dist','year','value', 'Flag']]],
            axis = 1)
    LMMAXT_HKP_3 = LMMAXT_HKP_3.rename(columns = {0: "Date"})
    
    
    # Information on the observed period by Observatory:
    Obs_summary['StartDate'] = LMMAXT_HKP_date.min()
    Obs_summary['EndDate'] = LMMAXT_HKP_date.max()
    
    Data_summary = pd.concat([Data_summary, Obs_summary], axis = 0)
    
    Agg_Data = pd.concat([Agg_Data, LMMAXT_HKP_3], axis = 0, ignore_index=True)

In [91]:
Data_summary

,Dist,NObs_Dist,NObs_Flagged,NObs_Starred,NObs_Str_Flagged,Starred_Pct(%),StartDate,EndDate
0,HKO,28552,1,0,0,0.00,1947-01-01,2025-02-28
0,TWN,6882,59,12,0,0.17,2006-05-01,2025-02-28
0,TW,5201,70,8,0,0.15,2010-12-07,2025-02-28
0,LFS,14399,349,1857,0,12.90,1985-10-01,2025-02-28
0,YCT,1068,3,1,0,0.09,2022-04-01,2025-02-28
0,TKL,13425,300,580,0,4.32,1988-06-01,2025-02-28
0,HKS,12999,298,455,0,3.50,1989-08-01,2025-02-28
0,TY1,5298,61,19,0,0.36,2010-09-01,2025-02-28
0,WLP,7033,92,29,0,0.41,2005-12-01,2025-02-28
0,PEN,7581,108,74,0,0.98,2004-06-01,2025-02-28


In [95]:
total_obs = Data_summary['NObs_Dist'].sum()
Data_Composition = pd.DataFrame({
    'Observatory':Data_summary['Dist'].values,
    'NObs':Data_summary['NObs_Dist'].values,
    'Percentage(%)To_TotalOBS':round((Data_summary['NObs_Dist']/total_obs)*100,2),
    'ObservedPeriod(#Year)':[np.ceil(td/np.timedelta64(1, 'D')).astype(int) for td in (Data_summary['EndDate']-Data_summary['StartDate'])/(30*12)] 
})

Data_Composition.sort_values(by=['Percentage(%)To_TotalOBS','ObservedPeriod(#Year)'], 
                             ascending = False,
                             inplace = True)
Data_Composition.reset_index(inplace=True, drop = True)

print(f"(1) NA value percentage (%): {round(100*Agg_Data.loc[Agg_Data['value'].isna()!=False].shape[0]/Agg_Data.shape[0],1)} % \n")
print(f"(2-1) Total number of observations (From Summary table sum-up): {total_obs}) \n")
print(f"(2-2) Total number of observations (From Aggregated data number of row counting): {Agg_Data.shape[0]} \n")
print(f"(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): {total_obs - Data_summary['NObs_Starred'].sum() -(3*24)} \n")
print("(4) Humidity data observed period: \n")
print(Data_Composition.head())
print(Data_Composition.tail())
print(f"\n (5) Humidity data mean observed period: {round(Data_Composition['ObservedPeriod(#Year)'].mean(),1)} year")

(1) NA value percentage (%): 0.0 % 

(2-1) Total number of observations (From Summary table sum-up): 249419) 

(2-2) Total number of observations (From Aggregated data number of row counting): 241237 

(3) The number of observations after filter application (From Summary table sum-up deducted the # of filtered lines): 241237 

(4) Humidity data observed period: 

  Observatory   NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
0         HKO  28552                     11.45                     80
1         SHA  14764                      5.92                     41
2         LFS  14399                      5.77                     40
3         TKL  13425                      5.38                     38
4         HKS  12999                      5.21                     37
   Observatory  NObs  Percentage(%)To_TotalOBS  ObservedPeriod(#Year)
19         TU1  6637                      2.66                     19
20         KSC  6059                      2.43                     17
21  

For Humidity data, the mean observed period of all the observatories is about 29 years, similar to Temperature data but slightly longer. The maximal observed period is 80 years, also for Hong Kong Observatory. 

In contrary, the percentage of null values (\*\*\*) are higher for Humidity dataset. Its maximal value is observed up to 17% (vs. 5.5% for Temperature dataset). The data quality, at least as of the starred observations, is also very good for the most historical observatory, Hong Kong Observatory. The other observatory which has 0 starred observation is Hong Kong International Airport (HKA).

Data Quality in general is worse for Humidity dataset, when it comes to the percentage of observatories having less than 1% of their dataset filled by starred lines. It only counts 50% of all the observatories, in contrary to Temperature dataset (71%).

In [102]:
print("(6) Missing values by Observatory: \n")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).head())
print("\n...\n")
print(Data_summary.iloc[:,[0,1,3,5]].sort_values(by='Starred_Pct(%)', ascending=False).tail())
print(f"\n (7) Percentage of observatories with less than 1% of missing values: {round(100*Data_summary.loc[Data_summary['Starred_Pct(%)']<1].shape[0]/Data_summary.shape[0],0)} %")


(6) Missing values by Observatory: 

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0  JKB      12147          2101           17.30
0  LFS      14399          1857           12.90
0  WGL      12968           872            6.72
0  SEK      10320           586            5.68
0  TKL      13425           580            4.32

...

  Dist  NObs_Dist  NObs_Starred  Starred_Pct(%)
0   TW       5201             8            0.15
0  YCT       1068             1            0.09
0   KP      11934             9            0.08
0  HKO      28552             0            0.00
0  HKA      10138             0            0.00

 (7) Percentage of observatories with less than 1% of missing values: 50.0 %


In [103]:
treated_data_rep = r'../0_Data/wrangled/' 
if not os.path.exists(treated_data_rep):
    os.makedirs(treated_data_rep)
Agg_Data.to_pickle(treated_data_rep+"Humidity_AGG.pkl")